## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Importing libraries
import pandas as pd
import sqlite3
import numpy as np

In [2]:
pd.options.display.max_columns = None

# Preparation of Dataset

## Accessing the different datasets

### Box Office Mojo Website

In [3]:

box_movie_gross_df = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
box_movie_gross_df.head(10)

# This dataset contains the domestic and foreign revenue generated by different movies from Box Office Movie Mojo

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How to Train Your Dragon,P/DW,217600000.0,277300000,2010


In [4]:
# unifying naming comvention in title column
box_movie_gross_df["title"] = box_movie_gross_df["title"].str.title()
box_movie_gross_df["title"] = box_movie_gross_df["title"].str.strip()

In [5]:
#Removing instances where date might be on the movie title and special characters
box_movie_gross_df['title'] = box_movie_gross_df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
box_movie_gross_df['title'] = box_movie_gross_df['title'].str.replace(r'[^\w\s]', '',regex=True)


In [6]:
box_movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [7]:
box_movie_gross_df.head(10)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice In Wonderland,BV,334200000.0,691300000,2010
2,Harry Potter And The Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How To Train Your Dragon,P/DW,217600000.0,277300000,2010


### Moviesdb Website

In [8]:

moviesdb_df =pd.read_csv("zippedData/tmdb.movies.csv.gz")

moviesdb_df.head(10)

#dataset contains moviesdb ratings of different titles


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
5,5,"[12, 14, 10751]",32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229
6,6,"[28, 12, 14, 878]",19995,en,Avatar,26.526,2009-12-18,Avatar,7.4,18676
7,7,"[16, 10751, 35]",10193,en,Toy Story 3,24.445,2010-06-17,Toy Story 3,7.7,8340
8,8,"[16, 10751, 35]",20352,en,Despicable Me,23.673,2010-07-09,Despicable Me,7.2,10057
9,9,"[16, 28, 35, 10751, 878]",38055,en,Megamind,22.855,2010-11-04,Megamind,6.8,3635


In [9]:
# converting all string to title case and stripped and removing special characters
moviesdb_df = moviesdb_df.applymap(lambda x:x.strip()  if isinstance(x,str)else x)
moviesdb_df = moviesdb_df.applymap(lambda x:x.title()  if isinstance(x,str)else x)
moviesdb_df['original_title'] = moviesdb_df['original_title'].str.replace(r'[^\w\s]', '',regex=True)
moviesdb_df['title'] = moviesdb_df['title'].str.replace(r'[^\w\s]', '',regex=True)

In [10]:
#renaming some columns for easier readability
moviesdb_df= moviesdb_df.rename(columns={"vote_average":"moviedb_average_rating", "vote_count":"moviedb_number_of_votes"})

In [11]:
moviesdb_df.head(10)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,moviedb_average_rating,moviedb_number_of_votes
0,0,"[12, 14, 10751]",12444,En,Harry Potter And The Deathly Hallows Part 1,33.533,2010-11-19,Harry Potter And The Deathly Hallows Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,En,How To Train Your Dragon,28.734,2010-03-26,How To Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,En,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,En,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,En,Inception,27.920,2010-07-16,Inception,8.3,22186
5,5,"[12, 14, 10751]",32657,En,Percy Jackson The Olympians The Lightning Thief,26.691,2010-02-11,Percy Jackson The Olympians The Lightning Thief,6.1,4229
6,6,"[28, 12, 14, 878]",19995,En,Avatar,26.526,2009-12-18,Avatar,7.4,18676
7,7,"[16, 10751, 35]",10193,En,Toy Story 3,24.445,2010-06-17,Toy Story 3,7.7,8340
8,8,"[16, 10751, 35]",20352,En,Despicable Me,23.673,2010-07-09,Despicable Me,7.2,10057
9,9,"[16, 28, 35, 10751, 878]",38055,En,Megamind,22.855,2010-11-04,Megamind,6.8,3635


### Rotten Tomato Website

In [12]:
rt_movie_info = pd.read_csv("zippedData/rt.movie_info.tsv.gz",sep='\t')
rt_movie_info.head(10)

#contains movie info from rotten tomatoes website

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
5,8,The year is 1942. As the Allies unite overseas...,PG,Drama|Kids and Family,Jay Russell,Gail Gilchriest,"Mar 3, 2000","Jul 11, 2000",NaN,NaN,95 minutes,Warner Bros. Pictures
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,"Jan 11, 2002","Jun 18, 2002",$,"41,032,915",82 minutes,Paramount Pictures
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"Apr 27, 2006","Oct 2, 2007",$,"224,114",123 minutes,Sony Pictures Classics
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,"134,904",117 minutes,NaN
9,15,When a diamond expedition in the Congo is lost...,PG-13,Action and Adventure|Mystery and Suspense|Scie...,Frank Marshall,John Patrick Shanley,"Jun 9, 1995","Jul 27, 1999",NaN,NaN,108 minutes,NaN


In [13]:
rt_reviews_df = pd.read_csv("zippedData/rt.reviews.tsv.gz",sep="\t",encoding="latin_1")
rt_reviews_df.head(10)


#contains reviews of various movies from the rotten tomato website

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
5,3,... Cronenberg's Cosmopolis expresses somethin...,NaN,fresh,Michelle Orange,0,Capital New York,"September 11, 2017"
6,3,"Quickly grows repetitive and tiresome, meander...",C,rotten,Eric D. Snider,0,EricDSnider.com,"July 17, 2013"
7,3,Cronenberg is not a director to be daunted by ...,2/5,rotten,Matt Kelemen,0,Las Vegas CityLife,"April 21, 2013"
8,3,"Cronenberg's cold, exacting precision and emot...",NaN,fresh,Sean Axmaker,0,Parallax View,"March 24, 2013"
9,3,Over and above its topical urgency or the bit ...,NaN,fresh,Kong Rithdee,0,Bangkok Post,"March 4, 2013"


### The Numbers Website

In [14]:
thenumbers_df =pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

thenumbers_df.head(10)

# contains the production budget and revenue of various movies sourced from the Numbers website

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
6,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200"
7,8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,"$300,000,000","$309,420,425","$963,420,425"
8,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209"
9,10,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923"


In [15]:
thenumbers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [16]:
# Removing special characters from the movie title name
thenumbers_df['movie'] = thenumbers_df['movie'].str.replace(r'[^\w\s]', '',regex=True)

In [17]:
# stripping the movie title and keeping them on title case
thenumbers_df['movie'] = thenumbers_df['movie'].str.strip()
thenumbers_df['movie'] = thenumbers_df['movie'].str.title()

In [18]:
thenumbers_df.head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates Of The Caribbean On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers Age Of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep Viii The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
5,6,"Dec 18, 2015",Star Wars Ep Vii The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
6,7,"Apr 27, 2018",Avengers Infinity War,"$300,000,000","$678,815,482","$2,048,134,200"
7,8,"May 24, 2007",Pirates Of The Caribbean At Worldâs End,"$300,000,000","$309,420,425","$963,420,425"
8,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209"
9,10,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923"


### IMDB website

In [19]:
conn=sqlite3.connect("zippedData/im.db/im.db")
cur= conn.cursor()

In [20]:
%%script sqlite3 zippedData/im.db/im.db --out tables
.tables
.quit

In [21]:
print(tables)

directors      movie_akas     movie_ratings  principals   
known_for      movie_basics   persons        writers      



In [22]:
query = """ 
SELECT 
    mb.movie_id,
    mb.primary_title,
    mb.original_title,
    mb.genres,
    mb.start_year AS release_year,
    mb.runtime_minutes AS film_runtime,
    mr.averagerating AS imdb_average_rating,
    mr.numvotes AS imdb_number_of_votes,
    GROUP_CONCAT(DISTINCT(p.primary_name)) AS directors
FROM movie_basics AS mb
LEFT JOIN movie_ratings AS mr ON mb.movie_id = mr.movie_id
JOIN
directors AS d
USING (movie_id)
JOIN
persons AS p
USING (person_id)
GROUP BY mb.movie_id;
"""

imdb_df=pd.read_sql(query,conn)

In [23]:
imdb_df.sample(20)

,movie_id,primary_title,original_title,genres,release_year,film_runtime,imdb_average_rating,imdb_number_of_votes,directors
84720,tt5069030,Journal de Bord d'un Bateau fantôme,Journal de Bord d'un Bateau fantôme,"Documentary,Drama,Fantasy",2012,110.0,NaN,NaN,Augusto Contento
84591,tt5061360,Promise at Dawn,La promesse de l'aube,"Biography,Drama,Romance",2017,131.0,7.2,1516.0,Eric Barbier
62354,tt3678938,Humpty Sharma Ki Dulhania,Humpty Sharma Ki Dulhania,"Comedy,Drama,Romance",2014,133.0,6.1,9835.0,Shashank Khaitan
12416,tt1679279,RU-486: The Last Option,RU-486: The Last Option,Drama,2011,89.0,7.3,19.0,Irene Maffei
40415,tt2501884,Seek,Seek,Drama,2014,76.0,4.5,76.0,Eric Henry
28237,tt2123262,Ordinary Miracles: The Photo League's New York,Ordinary Miracles: The Photo League's New York,Documentary,2012,74.0,9.4,9.0,"Daniel Allentuck,Nina Rosenblum"
133328,tt8946336,The Primrose,The Primrose,Comedy,2018,NaN,NaN,NaN,Jon Kilmer
10912,tt1637718,Society Kaam Se Gayi,Society Kaam Se Gayi,Drama,2011,NaN,6.2,10.0,Rahul Dholakia
128640,tt8413912,Quero-te Tanto!,Quero-te Tanto!,Comedy,2019,103.0,5.3,31.0,Vicente Alves do Ó
126906,tt8243286,Goalie,Goalie,"Biography,Drama,Sport",2019,111.0,7.2,23.0,Adriana Maggs


In [24]:
# converting strings into title case and stripped
imdb_df = imdb_df.applymap(lambda x:x.strip() if isinstance(x,str)else x)
imdb_df = imdb_df.applymap(lambda x:x.title() if isinstance(x,str)else x)
imdb_df['primary_title'] = imdb_df['primary_title'].str.replace(r'[^\w\s]', '',regex=True)
imdb_df['original_title'] = imdb_df['original_title'].str.replace(r'[^\w\s]', '',regex=True)

In [25]:
imdb_df.head(20)

,movie_id,primary_title,original_title,genres,release_year,film_runtime,imdb_average_rating,imdb_number_of_votes,directors
0,Tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",2013,175.0,7.0,77.0,Harnam Singh Rawail
1,Tt0066787,One Day Before The Rainy Season,Ashad Ka Ek Din,"Biography,Drama",2019,114.0,7.2,43.0,Mani Kaul
2,Tt0069049,The Other Side Of The Wind,The Other Side Of The Wind,Drama,2018,122.0,6.9,4517.0,Orson Welles
3,Tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",2018,NaN,6.1,13.0,Hrishikesh Mukherjee
4,Tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",2017,80.0,6.5,119.0,"Raoul Ruiz,Valeria Sarmiento"
5,Tt0111414,A Thin Life,A Thin Life,Comedy,2018,75.0,NaN,NaN,Frank Howson
6,Tt0112502,Bigfoot,Bigfoot,"Horror,Thriller",2017,NaN,4.1,32.0,Mc Jones
7,Tt0137204,Joe Finds Grace,Joe Finds Grace,"Adventure,Animation,Comedy",2017,83.0,8.1,263.0,Anthony Harrison
8,Tt0139613,O Silêncio,O Silêncio,"Documentary,History",2012,NaN,NaN,NaN,"José Manuel Alves Pereira,António Loja Neves"
9,Tt0144449,Nema Aviona Za Zagreb,Nema Aviona Za Zagreb,Biography,2012,82.0,NaN,NaN,Louis Van Gasteren


## Merging dataset

### merging imdb dataset to

In [26]:
# Merging the imdb website with the numbers website

merged_imdb_the_numbers = imdb_df.merge(thenumbers_df, left_on= "primary_title",right_on = "movie", how = "left")

In [27]:
# Merge the merged dataset to the moviedb website
merged_to_moviedb = merged_imdb_the_numbers.merge(moviesdb_df,left_on = "primary_title", right_on = "original_title", how = "left")

In [28]:
# Merge the merged dataset to the box office website
merged_to_box_office = merged_to_moviedb.merge(box_movie_gross_df, left_on = "primary_title", right_on="title", how = "left")

In [29]:
merged_to_box_office.head()

,movie_id,primary_title,original_title_x,genres,release_year,film_runtime,imdb_average_rating,imdb_number_of_votes,directors,id_x,release_date_x,movie,production_budget,domestic_gross_x,worldwide_gross,Unnamed: 0,genre_ids,id_y,original_language,original_title_y,popularity,release_date_y,title_x,moviedb_average_rating,moviedb_number_of_votes,title_y,studio,domestic_gross_y,foreign_gross,year
0,Tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",2013,175.0,7.0,77.0,Harnam Singh Rawail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tt0066787,One Day Before The Rainy Season,Ashad Ka Ek Din,"Biography,Drama",2019,114.0,7.2,43.0,Mani Kaul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tt0069049,The Other Side Of The Wind,The Other Side Of The Wind,Drama,2018,122.0,6.9,4517.0,Orson Welles,NaN,NaN,NaN,NaN,NaN,NaN,24185.0,"[35, 18]",299782.0,En,The Other Side Of The Wind,9.8,2018-11-02,The Other Side Of The Wind,7.0,64.0,NaN,NaN,NaN,NaN,NaN
3,Tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",2018,NaN,6.1,13.0,Hrishikesh Mukherjee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",2017,80.0,6.5,119.0,"Raoul Ruiz,Valeria Sarmiento",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
merged_to_box_office.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144696 entries, 0 to 144695
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   movie_id                 144696 non-null  object 
 1   primary_title            144696 non-null  object 
 2   original_title_x         144694 non-null  object 
 3   genres                   141464 non-null  object 
 4   release_year             144696 non-null  int64  
 5   film_runtime             115836 non-null  float64
 6   imdb_average_rating      76158 non-null   float64
 7   imdb_number_of_votes     76158 non-null   float64
 8   directors                144696 non-null  object 
 9   id_x                     5286 non-null    float64
 10  release_date_x           5286 non-null    object 
 11  movie                    5286 non-null    object 
 12  production_budget        5286 non-null    object 
 13  domestic_gross_x         5286 non-null    object 
 14  worl

In [31]:
#dropping duplicated columns caused by merging of the dataset
merged_to_box_office=merged_to_box_office.drop(["movie_id","original_title_x","id_x","movie","Unnamed: 0","genre_ids","id_y","original_title_y","title_x","title_y"], axis = 1)

In [32]:
merged_to_box_office.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144696 entries, 0 to 144695
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   primary_title            144696 non-null  object 
 1   genres                   141464 non-null  object 
 2   release_year             144696 non-null  int64  
 3   film_runtime             115836 non-null  float64
 4   imdb_average_rating      76158 non-null   float64
 5   imdb_number_of_votes     76158 non-null   float64
 6   directors                144696 non-null  object 
 7   release_date_x           5286 non-null    object 
 8   production_budget        5286 non-null    object 
 9   domestic_gross_x         5286 non-null    object 
 10  worldwide_gross          5286 non-null    object 
 11  original_language        22703 non-null   object 
 12  popularity               22703 non-null   float64
 13  release_date_y           22703 non-null   object 
 14  movi

In [33]:
# dealing with date columns
# release_year
merged_to_box_office['release_year'] = pd.to_datetime(merged_to_box_office['release_year'], format='%Y')
# release_date_x
merged_to_box_office['release_date_x'] = pd.to_datetime(merged_to_box_office['release_date_x'],format='%b %d, %Y' ,yearfirst= True, errors= 'raise')
#release_date_y
merged_to_box_office['release_date_y'] = pd.to_datetime(merged_to_box_office['release_date_y'],format = "%Y-%m-%d", yearfirst= True,errors = "raise")
#year
merged_to_box_office['year'] = pd.to_datetime(merged_to_box_office['year'],format='%Y')



In [34]:
#creating a column that will store the year a movie released
merged_to_box_office['release_year'] = merged_to_box_office['release_year'].dt.year



In [35]:
# creating a column that will store the month a movie was released
merged_to_box_office['release_month'] = merged_to_box_office['release_date_x'].dt.month_name()

In [36]:
#filling null values in release_year
merged_to_box_office['release_year'] = merged_to_box_office['release_year'].fillna(merged_to_box_office['release_date_x'].dt.year).fillna(merged_to_box_office['release_date_y'].dt.year).fillna(merged_to_box_office['year'].dt.year)

#filling null values in release_month
merged_to_box_office['release_month'] = merged_to_box_office['release_month'].fillna(merged_to_box_office['release_date_y'].dt.month_name())

In [37]:
# Dropping the extra date columns

merged_to_box_office=merged_to_box_office.drop(["release_date_x","release_date_y","year"], axis= 1)

In [38]:
merged_to_box_office.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144696 entries, 0 to 144695
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   primary_title            144696 non-null  object 
 1   genres                   141464 non-null  object 
 2   release_year             144696 non-null  int64  
 3   film_runtime             115836 non-null  float64
 4   imdb_average_rating      76158 non-null   float64
 5   imdb_number_of_votes     76158 non-null   float64
 6   directors                144696 non-null  object 
 7   production_budget        5286 non-null    object 
 8   domestic_gross_x         5286 non-null    object 
 9   worldwide_gross          5286 non-null    object 
 10  original_language        22703 non-null   object 
 11  popularity               22703 non-null   float64
 12  moviedb_average_rating   22703 non-null   float64
 13  moviedb_number_of_votes  22703 non-null   float64
 14  stud

In [39]:
#changed the currency columns from string datatype to float
merged_to_box_office['production_budget'] = merged_to_box_office['production_budget'].replace({'\$': '', ',': ''}, regex=True).astype(float)
merged_to_box_office['domestic_gross_x']= merged_to_box_office['domestic_gross_x'].replace({'\$': '', ',': ''}, regex=True).astype(float)
merged_to_box_office['foreign_gross'] = merged_to_box_office['foreign_gross'].replace({'\$': '', ',': ''},regex= True).astype(float)
merged_to_box_office['worldwide_gross']= merged_to_box_office['worldwide_gross'].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [40]:
#unifying the domestic gross column
merged_to_box_office['domestic_gross']=merged_to_box_office['domestic_gross_x']
merged_to_box_office['domestic_gross']=merged_to_box_office['domestic_gross'].fillna(merged_to_box_office['domestic_gross_y'])

#unifying the foreign gross column

merged_to_box_office["worldwide_gross"]= merged_to_box_office['worldwide_gross'].fillna(merged_to_box_office['foreign_gross'])


In [41]:
#dropping duplicated currency columns
merged_to_box_office = merged_to_box_office.drop(['domestic_gross_x','domestic_gross_y','foreign_gross'],axis = 1)

In [42]:
merged_to_box_office.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144696 entries, 0 to 144695
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   primary_title            144696 non-null  object 
 1   genres                   141464 non-null  object 
 2   release_year             144696 non-null  int64  
 3   film_runtime             115836 non-null  float64
 4   imdb_average_rating      76158 non-null   float64
 5   imdb_number_of_votes     76158 non-null   float64
 6   directors                144696 non-null  object 
 7   production_budget        5286 non-null    float64
 8   worldwide_gross          6340 non-null    float64
 9   original_language        22703 non-null   object 
 10  popularity               22703 non-null   float64
 11  moviedb_average_rating   22703 non-null   float64
 12  moviedb_number_of_votes  22703 non-null   float64
 13  studio                   6000 non-null    object 
 14  rele

In [43]:
columns = merged_to_box_office.columns.to_list()
columns

['primary_title',
 'genres',
 'release_year',
 'film_runtime',
 'imdb_average_rating',
 'imdb_number_of_votes',
 'directors',
 'production_budget',
 'worldwide_gross',
 'original_language',
 'popularity',
 'moviedb_average_rating',
 'moviedb_number_of_votes',
 'studio',
 'release_month',
 'domestic_gross']

In [44]:
columns_reordered= ['primary_title',
 'genres',
 'directors',
 'studio',
 'release_year',
 'release_month',
 'film_runtime',
 'imdb_average_rating',
 'imdb_number_of_votes',
 'production_budget',
 'domestic_gross',
 'worldwide_gross',
 'original_language',
 'popularity',
 'moviedb_average_rating',
 'moviedb_number_of_votes']

merged_to_box_office = merged_to_box_office[columns_reordered]
merged_to_box_office.head()


,primary_title,genres,directors,studio,release_year,release_month,film_runtime,imdb_average_rating,imdb_number_of_votes,production_budget,domestic_gross,worldwide_gross,original_language,popularity,moviedb_average_rating,moviedb_number_of_votes
0,Sunghursh,"Action,Crime,Drama",Harnam Singh Rawail,NaN,2013,NaN,175.0,7.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,One Day Before The Rainy Season,"Biography,Drama",Mani Kaul,NaN,2019,NaN,114.0,7.2,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Other Side Of The Wind,Drama,Orson Welles,NaN,2018,November,122.0,6.9,4517.0,NaN,NaN,NaN,En,9.8,7.0,64.0
3,Sabse Bada Sukh,"Comedy,Drama",Hrishikesh Mukherjee,NaN,2018,NaN,NaN,6.1,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Wandering Soap Opera,"Comedy,Drama,Fantasy","Raoul Ruiz,Valeria Sarmiento",NaN,2017,NaN,80.0,6.5,119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
merged_dataset = merged_to_box_office.copy()

In [47]:
conn.close()